In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [ ]:
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()

In [ ]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')

In [ ]:
sample_0 = train_df[train_df["actual"] == 0].sample(n=50, random_state=42)
sample_1 = train_df[train_df["actual"] == 1].sample(n=50, random_state=42)
sampled_train_df = pd.concat([sample_0, sample_1]).sample(frac=1, random_state=42).reset_index(drop=True)
sampled_train_df

In [ ]:
sample_0 = test_df[test_df["actual"] == 0].sample(n=50, random_state=42)
sample_1 = test_df[test_df["actual"] == 1].sample(n=30, random_state=42)
sampled_test_df = pd.concat([sample_0, sample_1]).sample(frac=1, random_state=42).reset_index(drop=True)
sampled_test_df

In [ ]:
train_spark_df = spark.createDataFrame(sampled_train_df)
test_spark_df = spark.createDataFrame(sampled_test_df)

## Plot 1

In [ ]:
import numpy as np
import pandas as pd
import math
from pyspark.sql.functions import col
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import sample_colorscale
from typing import List, Dict, Optional

# Configuration for bar color mapping: invert True for metrics where lower is better
METRICS_CONFIG = {
    "accuracy":           {"invert": False},
    "precision":          {"invert": False},
    "recall":             {"invert": False},
    "f1":                 {"invert": False},
    "specificity":        {"invert": False},
    "npv":                {"invert": False},
    "balanced_accuracy":  {"invert": False},
    "jaccard":            {"invert": False},
    "gmean":              {"invert": False},
    "hamming_loss":       {"invert": True}  # lower is better
}

CM_X = ["Predicted '0'", "Predicted '1'"]
CM_Y = ["Actual '0'", "Actual '1'"]

def _compute_threshold_metrics(df_spark, label_col: str, probability_col: str,
                               thresholds: List[float], extra_metrics: bool = False) -> pd.DataFrame:
    """
    Compute confusion matrix counts and classification metrics across given thresholds.

    Parameters:
        df_spark: Input Spark DataFrame.
        label_col (str): Column name for the true label.
        probability_col (str): Column name for prediction probability.
        thresholds (List[float]): List of threshold values to compute metrics.
        extra_metrics (bool): Whether to compute additional metrics.

    Returns:
        pd.DataFrame: DataFrame containing confusion matrix counts and metrics for each threshold.
    """
    # Cast probability and label columns to double for calculation consistency.
    df = df_spark.withColumn("score", col(probability_col).cast("double"))
    df = df.withColumn("label_d", col(label_col).cast("double"))

    rows = []
    for t in thresholds:
        # Compute prediction based on threshold
        pred = df.withColumn("pred_t", (col("score") >= t).cast("double"))
        # Create confusion matrix using pivot operation and fill missing counts with zero
        cm = (pred.groupBy("label_d")
                  .pivot("pred_t", [0.0, 1.0])
                  .count().na.fill(0).orderBy("label_d").collect())
        # Extract TN, FP, FN, TP counts from confusion matrix
        TN, FP = cm[0][1], cm[0][2]
        FN, TP = cm[1][1], cm[1][2]
        total = TP + TN + FP + FN

        # Calculate primary metrics: accuracy, precision, recall, and f1 score.
        acc  = (TP + TN) / total if total else 0.0
        prec = TP / (TP + FP)    if (TP + FP) else 0.0
        rec  = TP / (TP + FN)    if (TP + FN) else 0.0
        f1   = (2 * prec * rec) / (prec + rec) if (prec + rec) else 0.0

        row = {"threshold": t, "TN": TN, "FP": FP, "FN": FN, "TP": TP,
               "accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

        # If extra_metrics is True, calculate additional metrics.
        if extra_metrics:
            spec = TN / (TN + FP) if (TN + FP) else 0.0
            npv  = TN / (TN + FN) if (TN + FN) else 0.0
            bal_acc = (rec + spec) / 2
            jaccard = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0
            gmean   = math.sqrt(rec * spec) if rec * spec >= 0 else 0.0
            h_loss  = (FP + FN) / total if total else 0.0
            row.update({"specificity": spec, "npv": npv,
                        "balanced_accuracy": bal_acc,
                        "jaccard": jaccard, "gmean": gmean,
                        "hamming_loss": h_loss})

        rows.append(row)

    return pd.DataFrame(rows)

def _metric_color(value: float, invert: bool) -> str:
    """
    Map metric value to a color on the RdYlGn scale.

    Parameters:
        value (float): The metric value.
        invert (bool): Whether to invert the value (for metrics where lower is better).

    Returns:
        str: The color corresponding to the metric value.
    """
    # Invert the value if required for color scaling
    scaled = 1 - value if invert else value
    return sample_colorscale('RdYlGn', [scaled])[0]

def _create_slider_steps(df_metrics: pd.DataFrame, metrics: List[str]) -> List[Dict]:
    """
    Generate Plotly slider steps for each threshold, updating confusion matrix and metrics.

    Parameters:
        df_metrics (pd.DataFrame): DataFrame containing metrics for each threshold.
        metrics (List[str]): List of metric names to be displayed in the bar chart.

    Returns:
        List[Dict]: List of slider step configurations.
    """
    steps = []
    for _, r in df_metrics.iterrows():
        # Build confusion matrix from metric row
        z = [[r.TN, r.FP], [r.FN, r.TP]]
        # Collect metric values for the bar chart
        vals = [r[m] for m in metrics]
        # Determine color for each metric bar based on its value and configuration
        bar_colors = [_metric_color(r[m], METRICS_CONFIG[m]["invert"]) for m in metrics]
        bar_text = [f"{v:.2f}" for v in vals]

        steps.append({
            'method': 'update',
            'args': [
                {
                    'z': [z, None],
                    'x': [CM_X, vals],
                    'marker.color': [None, bar_colors],
                    'text': [z, bar_text]
                },
                {'transition': {'duration': 500, 'easing': 'linear'}}
            ],
            'label': f"{r.threshold:.2f}"
        })
    return steps

def _configure_slider(steps: List[Dict], active_index: int) -> Dict:
    return {
        "active": active_index,
        "pad": {"t": 50},
        "len": 0.8,
        "x": 0.1,
        "steps": steps,
        "currentvalue": {
            "visible": True,
            "prefix": "Threshold: ",
            "xanchor": "center",
            "font": {"size": 14, "color": "black"},
        },
        # hide every tick-mark
        "ticklen":   0,
        "tickwidth": 0,
        "tickcolor": "rgba(0,0,0,0)",
        "font": {"color": "rgba(0,0,0,0)"},  # still hide the labels
        "bgcolor": "white",
        "bordercolor": "lightgray",
    }


def _initialize_figure(initial_z: List[List[int]], initial_vals: List[float], metrics: List[str]) -> go.Figure:
    """
    Initialize a Plotly figure with a confusion matrix heatmap and a metrics bar chart.

    Parameters:
        initial_z (List[List[int]]): Initial confusion matrix counts.
        initial_vals (List[float]): Initial metric values for the bar chart.
        metrics (List[str]): Metrics to be displayed.

    Returns:
        go.Figure: Initialized Plotly figure.
    """
    # Create subplots with two columns: one for confusion matrix and one for metrics
    fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                        subplot_titles=("Confusion Matrix", "Metrics"))

    # Add confusion matrix heatmap on the left subplot
    fig.add_trace(go.Heatmap(
        z=initial_z, x=CM_X, y=CM_Y,
        text=initial_z, texttemplate="%{text}", showscale=False,
        colorscale="Greens"
    ), row=1, col=1)

    # Calculate bar colors for the metrics
    bar_colors = [_metric_color(val, METRICS_CONFIG[m]["invert"]) for val, m in zip(initial_vals, metrics)]
    # Add horizontal bar chart on the right subplot
    fig.add_trace(go.Bar(
        y=metrics, x=initial_vals, orientation='h',
        marker_color=bar_colors,
        text=[f"{v:.2f}" for v in initial_vals], textposition='auto'
    ), row=1, col=2)

    return fig

def _style_figure(fig: go.Figure, slider: Dict) -> None:
    """
    Apply layout styles, axis formatting, and font styling to the Plotly figure.

    Parameters:
        fig (go.Figure): The Plotly figure to be styled.
        slider (Dict): The slider configuration to be applied.
    """
    # Update global layout attributes
    fig.update_layout(
        sliders=[slider],
        transition={'duration': 500, 'easing': 'linear'},
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'family': 'sans-serif', 'size': 12, 'color': '#333'}
    )
    # Configure x and y axes for both subplots
    fig.update_xaxes(showgrid=False, ticklabelstandoff=10, tickfont=dict(size=12, family='sans-serif'))
    fig.update_yaxes(showgrid=False, ticklabelstandoff=10, tickfont=dict(size=12, family='sans-serif'))
    fig.update_xaxes(tickmode='array', tickvals=CM_X, type='category', row=1, col=1)
    fig.update_yaxes(tickmode='array', tickvals=CM_Y, type='category', row=1, col=1)
    fig.update_xaxes(autorange=False, showticklabels=False, range=[1, 0], row=1, col=2)
    fig.update_yaxes(side='right', row=1, col=2)

    # Enhance subplot titles appearance
    for ann in fig.layout.annotations:
        ann.text = f"<b>{ann.text}</b>"
        ann.y += 0.05
        ann.font = {'family': 'sans-serif', 'size': 16, 'color': 'DarkSlateGray'}

def plot_metrics_confusion_interactive(
    spark_df,
    label_col: str = 'label',
    probability_col: str = 'probability',
    thresholds: Optional[List[float]] = None,
    default_threshold: float = 0.50,
    extra_metrics: bool = False
) -> go.Figure:
    """
    Build an interactive Plotly figure that displays a confusion matrix and
    classification metrics with a slider to animate over different thresholds.

    Parameters:
        spark_df: Input Spark DataFrame.
        label_col (str): Name of the column containing true labels.
        probability_col (str): Name of the column containing probability scores.
        thresholds (Optional[List[float]]): List of threshold values. Defaults to a range from 0.0 to 1.0.
        default_threshold (float): The default active threshold value.
        extra_metrics (bool): Whether to compute and display extra metrics.

    Returns:
        go.Figure: The interactive Plotly figure.
    """
    # Define default thresholds if not provided
    if thresholds is None:
        thresholds = np.arange(0.0, 1.01, 0.05).tolist()
    # Find the index of the default threshold if present
    default_idx = thresholds.index(default_threshold) if default_threshold in thresholds else 0

    # Compute metrics for each threshold using the private helper function
    dfm = _compute_threshold_metrics(spark_df, label_col, probability_col, thresholds, extra_metrics)

    # Define main and extra metrics based on the extra_metrics flag
    base = ['accuracy', 'precision', 'recall', 'f1']
    extra = ['specificity', 'npv', 'balanced_accuracy', 'jaccard', 'gmean', 'hamming_loss'] if extra_metrics else []
    metrics = base + extra

    # Get initial data from the default threshold row
    init = dfm.iloc[default_idx]
    initial_z = [[init.TN, init.FP], [init.FN, init.TP]]
    initial_vals = [init[m] for m in metrics]

    # Initialize the figure, create the slider steps, and apply styling
    fig = _initialize_figure(initial_z, initial_vals, metrics)
    steps = _create_slider_steps(dfm, metrics)
    slider = _configure_slider(steps, default_idx)
    _style_figure(fig, slider)

    return fig, dfm


In [ ]:
train_fig, train_eval_df = plot_metrics_confusion_interactive(
    train_spark_df,
    label_col="actual",
    extra_metrics=True,
    thresholds=np.arange(0.0, 1.01, 0.01).tolist()
)

In [ ]:
test_fig, test_eval_df = plot_metrics_confusion_interactive(
    test_spark_df,
    label_col="actual",
    extra_metrics=True,
    thresholds=np.arange(0.0, 1.01, 0.01).tolist()
)

In [ ]:
train_fig

In [ ]:
test_fig

## Plot 2

In [ ]:
import pandas as pd
import plotly.express as px
from typing import List

def plot_interactive_scatter(df: pd.DataFrame, label_col: str = "label"):
    """
    Create an interactive scatter plot with dropdowns to select X and Y axes,
    coloring the points by the specified label column.
    
    Parameters:
    - df: Input Pandas DataFrame
    - label_col: Name of the label column for color grouping
    """
    # Identify numeric columns excluding the label
    numeric_cols: List[str] = df.select_dtypes(include='number').columns.drop(label_col).tolist()

    # Create base figure (first pair of columns)
    fig = px.scatter(df, x=numeric_cols[0], y=numeric_cols[1], color=label_col,
                     title="Interactive Scatter Plot by Label",
                     labels={label_col: label_col})

    # Dropdown for x-axis
    x_dropdown = [
        dict(label=col, method="update",
             args=[{"x": [df[col]]},
                   {"xaxis": {"title": col}}])
        for col in numeric_cols
    ]

    # Dropdown for y-axis
    y_dropdown = [
        dict(label=col, method="update",
             args=[{"y": [df[col]]},
                   {"yaxis": {"title": col}}])
        for col in numeric_cols
    ]

    # Update layout with dropdown menus
    fig.update_layout(
        updatemenus=[
            dict(buttons=x_dropdown, direction="down", showactive=True,
                 x=0.0, y=1.15, xanchor="left", yanchor="top",
                 pad={"r": 10, "t": 10},
                 name="X Axis"),
            dict(buttons=y_dropdown, direction="down", showactive=True,
                 x=0.25, y=1.15, xanchor="left", yanchor="top",
                 pad={"r": 10, "t": 10},
                 name="Y Axis")
        ]
    )

    # Style and return
    fig.update_layout(
        plot_bgcolor="white",
        paper_bgcolor="white",
        font=dict(family="sans-serif", size=12, color="#333")
    )
    return fig


In [ ]:
plot_interactive_scatter(
    train_df,
    label_col="actual"
)

## Plot 3

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_feature_distribution_with_prediction(
    df: pd.DataFrame,
    feature: str,
    label_col: str = "label",
    probability_col: str = "probability",
    threshold_values: np.ndarray = np.arange(0.0, 1.01, 0.05)
):
    """
    Plots side-by-side KDE plots of a numerical feature for:
    - Actual labels (static)
    - Predicted labels (interactive with threshold slider)

    Parameters:
    - df: DataFrame with actual and predicted probability columns
    - feature: feature column to plot
    - label_col: name of actual label column
    - probability_col: name of predicted probability column
    - threshold_values: list of thresholds to animate predictions
    """
    if feature not in df.columns or label_col not in df.columns or probability_col not in df.columns:
        raise ValueError("One or more columns not found in the DataFrame")

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Actual Labels", "Predicted Labels"),
        column_widths=[0.5, 0.5]
    )

    # --- Static: Actual KDEs ---
    actual_labels = sorted(df[label_col].dropna().unique())
    for label in actual_labels:
        data = df[df[label_col] == label][feature].dropna()
        if len(data) > 1:
            kde = gaussian_kde(data)
            x_vals = np.linspace(data.min(), data.max(), 100)
            y_vals = kde(x_vals)
            fig.add_trace(
                go.Scatter(
                    x=x_vals,
                    y=y_vals,
                    mode='lines',
                    name=f"Actual {label}",
                    fill='tozeroy',
                    legendgroup=f"actual_{label}"
                ),
                row=1, col=1
            )

    # --- Static: First threshold predictions ---
    t0 = threshold_values[0]
    df["pred_label"] = (df[probability_col] >= t0).astype(int)
    pred_labels = sorted(df["pred_label"].unique())

    for label in pred_labels:
        data = df[df["pred_label"] == label][feature].dropna()
        if len(data) > 1:
            kde = gaussian_kde(data)
            x_vals = np.linspace(data.min(), data.max(), 100)
            y_vals = kde(x_vals)
            fig.add_trace(
                go.Scatter(
                    x=x_vals,
                    y=y_vals,
                    mode='lines',
                    name=f"Pred {label}",
                    fill='tozeroy',
                    legendgroup=f"pred_{label}"
                ),
                row=1, col=2
            )

    # --- Build frames for predicted side ---
    frames = []
    for t in threshold_values:
        df["pred_label"] = (df[probability_col] >= t).astype(int)
        traces = []
        for label in sorted(df["pred_label"].unique()):
            data = df[df["pred_label"] == label][feature].dropna()
            if len(data) > 1:
                kde = gaussian_kde(data)
                x_vals = np.linspace(data.min(), data.max(), 100)
                y_vals = kde(x_vals)
                traces.append(go.Scatter(
                    x=x_vals,
                    y=y_vals,
                    mode='lines',
                    name=f"Pred {label}",
                    fill='tozeroy',
                    legendgroup=f"pred_{label}",
                    showlegend=False
                ))
        frames.append(go.Frame(data=traces, name=f"{t:.2f}"))

    fig.frames = frames

    # --- Slider ---
    steps = [{
        "method": "animate",
        "args": [[f"{t:.2f}"], {"mode": "immediate", "frame": {"duration": 500, "redraw": True}}],
        "label": f"{t:.2f}"
    } for t in threshold_values]

    fig.update_layout(
        title=f"Feature Distribution: {feature}",
        sliders=[{
            "steps": steps,
            "active": 0,
            "x": 0.1,
            "len": 0.8,
            "pad": {"t": 50},
            "currentvalue": {
                "visible": True,
                "prefix": "Threshold: ",
                "xanchor": "center",
                "font": {"size": 14}
            },
        }],
        plot_bgcolor="white",
        paper_bgcolor="white",
        font={"family": "sans-serif", "size": 12, "color": "#333"}
    )

    fig.show()


In [ ]:
feature_cols: List[str] = df.columns.drop(label_col).tolist()
total_rows = len(df)

# Determine feature types based on the provided definition
feature_types = {}
for col in feature_cols:
    unique_ratio = (df[col].nunique() / total_rows) * 100
    feature_types[col] = 'numerical' if unique_ratio > 1 else 'categorical'

In [ ]:
for col in train_df.columns:
    print(f"{col}: {train_df[col].nunique()} unique values")

In [ ]:
plot_feature_distribution_with_prediction(train_df,"EstimatedSalary", "actual")

In [ ]:
plot_feature_distribution(train_df,"Age", "actual")

In [ ]:
plot_feature_distribution(train_df,"Tenure", "actual")

## Plot 4

In [ ]:
import numpy as np
import plotly.graph_objects as go

# ─── Helpers ────────────────────────────────────────────────────────────────

def compute_auc(x: np.ndarray, y: np.ndarray) -> float:
    """Simple trapezoidal AUC."""
    return np.trapz(y, x)

def make_zone_polygons(random_fpr: np.ndarray, random_tpr: np.ndarray):
    """Return two dicts of kwargs for the red (below) and green (above) fill polygons."""
    red = dict(
        x=np.concatenate([random_fpr, random_fpr[::-1]]),
        y=np.concatenate([random_tpr, np.zeros_like(random_tpr)]),
        fill='toself', fillcolor='rgba(255,0,0,0.2)', line=dict(width=0),
        hoverinfo='skip', showlegend=False
    )
    green = dict(
        x=np.concatenate([random_fpr, random_fpr[::-1]]),
        y=np.concatenate([random_tpr, np.ones_like(random_tpr)]),
        fill='toself', fillcolor='rgba(0,128,0,0.2)', line=dict(width=0),
        hoverinfo='skip', showlegend=False
    )
    return red, green

# ─── Main plotting function ─────────────────────────────────────────────────

def plot_reference_rocs():
    # 1) Define random diagonal
    random_fpr = np.linspace(0, 1, 200)
    random_tpr = random_fpr

    # 2) Define curves and compute their AUCs
    curves = [
        {
            "name": "Random",
            "x": random_fpr, "y": random_tpr,
            "line": dict(dash='dash', color='black', width=2),
            "annot_x": 0.4, "annot_y": 0.4,
            "annot_offset": (0.4, 0.55),
            "annot_color": "black",
            "annot_text": "AUC=0.50 (Random)",
            "textangle": 0
        },
        {
            "name": "Perfect",
            "x": np.array([0, 0, 1]), "y": np.array([0, 1, 1]),
            "line": dict(color='green', width=3),
            "annot_x": 0.2, "annot_y": 1.0,
            "annot_offset": (0.2, 1.1),
            "annot_color": "green",
            "annot_text": "AUC=1.00 (Perfect)",
            "textangle": 0
        },
        {
            "name": "Good",
            "x": np.linspace(0, 1, 200),
            "y": None,  # to be filled below
            "line": dict(color='blue', width=3),
            "annot_x": 0.3, "annot_y": None,
            "annot_offset": (0.3, 0.8),
            "annot_color": "blue",
            "annot_text": "AUC ∈ (0.5, 1) (Good)",
            "textangle": 0
        },
        {
            "name": "Poor",
            "x": np.linspace(0, 1, 200),
            "y": None,
            "line": dict(color='red', width=3),
            "annot_x": 0.5, "annot_y": None,
            "annot_offset": (0.5, 0.3),
            "annot_color": "red",
            "annot_text": " AUC ∈ (0, 0.5) (Poor)",
            "textangle": 0
        }
    ]

    # Populate Good & Poor curves with data + AUC + annotation props
    for c in curves:
        if c["name"] == "Good":
            c["y"] = np.cbrt(c["x"])
            auc = compute_auc(c["x"], c["y"])
            c["annot_y"] = np.cbrt(c["annot_x"])
        elif c["name"] == "Poor":
            c["y"] = c["x"]**3
            auc = compute_auc(c["x"], c["y"])
            c["annot_y"] = c["annot_x"]**3

    # 3) Build figure
    fig = go.Figure()

    # 3a) Add zones
    red_zone, green_zone = make_zone_polygons(random_fpr, random_tpr)
    fig.add_trace(go.Scatter(**red_zone))
    fig.add_trace(go.Scatter(**green_zone))

    # 3b) Add all curves
    for c in curves:
        fig.add_trace(go.Scatter(
            x=c["x"], y=c["y"], mode='lines',
            line=c["line"],
            showlegend=False
        ))

    # 4) Add annotations
    for c in curves:
        fig.add_annotation(
            x=c["annot_x"], y=c["annot_y"],
            ax=c["annot_offset"][0], ay=c["annot_offset"][1],
            xref="x", yref="y", axref="x", ayref="y",
            text=c["annot_text"],
            showarrow=True, arrowhead=2, arrowsize=1,
            arrowcolor=c["annot_color"], arrowwidth=1,
            textangle=c["textangle"],
            font=dict(color=c["annot_color"], size=14)
        )

    # 5) Final layout
    fig.update_layout(
        title="Reference ROC Curves with On‐Curve AUC Labels",
        xaxis_title="False Positive Rate (1 – Specificity)",
        yaxis_title="True Positive Rate (Recall)",
        plot_bgcolor="white", paper_bgcolor="white",
        width=700, height=600
    )

    return fig



In [ ]:
plot_reference_rocs()

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd

def plot_roc_curve_area(train_eval_df: pd.DataFrame,
                        test_eval_df: pd.DataFrame) -> go.Figure:
    """
    Overlay smooth ROC areas for train and test, hide their legend entries,
    and place the AUC values inside the plot:
      - if AUC > 0.5 → bottom-right,
      - if AUC < 0.5 → top-left.
    """
    def prepare(df):
        dfc = df.copy()
        dfc["fpr"] = dfc["FP"] / (dfc["FP"] + dfc["TN"])
        dfc["tpr"] = dfc["TP"] / (dfc["TP"] + dfc["FN"])
        dfc = dfc.sort_values("fpr")
        auc = np.trapz(dfc["tpr"].values, dfc["fpr"].values)
        return dfc, auc

    def pos_for_auc(auc: float):
        # bottom-right if >0.5, else top-left
        if auc > 0.5:
            return dict(x=0.95, y=0.05, xanchor="right", yanchor="bottom")
        else:
            return dict(x=0.05, y=0.95, xanchor="left",  yanchor="top")

    # prepare both
    df_train, auc_train = prepare(train_eval_df)
    df_test,  auc_test  = prepare(test_eval_df)

    fig = go.Figure()

    # ---- Train ROC trace ----
    fig.add_trace(go.Scatter(
        x=df_train["fpr"], y=df_train["tpr"],
        mode="lines",
        name="Train ROC",
        hovertemplate=(
            "Train ROC<br>"
            "Threshold %{text}<br>"
            "FPR = %{x:.2f}<br>"
            "TPR = %{y:.2f}<extra></extra>"
        ),
        text=[f"{t:.2f}" for t in df_train["threshold"]],
        line=dict(color="rgb(31,119,180)", shape="spline", smoothing=1.3),
        fill="tozeroy", fillcolor="rgba(31,119,180,0.2)",
        marker=dict(size=6),
        showlegend=False
    ))

    # ---- Test ROC trace ----
    fig.add_trace(go.Scatter(
        x=df_test["fpr"], y=df_test["tpr"],
        mode="lines",
        name="Test ROC",
        hovertemplate=(
            "Test ROC<br>"
            "Threshold %{text}<br>"
            "FPR = %{x:.2f}<br>"
            "TPR = %{y:.2f}<extra></extra>"
        ),
        text=[f"{t:.2f}" for t in df_test["threshold"]],
        line=dict(color="rgb(255,127,14)", shape="spline", smoothing=1.3),
        fill="tozeroy", fillcolor="rgba(255,127,14,0.2)",
        marker=dict(size=6),
        showlegend=False
    ))

    # Diagonal chance
    fig.add_shape(
        type="line", x0=0, y0=0, x1=1, y1=1,
        line=dict(dash="dash", color="gray")
    )

    # Place train AUC
    p_train = pos_for_auc(auc_train)
    fig.add_annotation(
        text=f"<b>Train AUC:</b> {auc_train:.2f}",
        showarrow=False,
        font=dict(color="rgb(31,119,180)", size=14),
        bgcolor="rgba(255,255,255,0.7)",
        xref="paper", yref="paper",
        **p_train
    )

    # Place test AUC
    p_test = pos_for_auc(auc_test)
    # shift slightly if they collide
    if p_test == p_train:
        # if both on same corner, offset the test one inward
        p_test['x'] -= 0.0 if p_test['xanchor']=="right" else -0.0
        p_test['y'] += 0.07 if p_test['yanchor']=="bottom" else -0.07

    fig.add_annotation(
        text=f"<b>Test AUC:</b> {auc_test:.2f}",
        showarrow=False,
        font=dict(color="rgb(255,127,14)", size=14),
        bgcolor="rgba(255,255,255,0.7)",
        xref="paper", yref="paper",
        **p_test
    )

    fig.update_layout(
        title={
            "text": "Train vs. Test ROC Curves",
            "x": 0.5, "xanchor": "center"
        },
        title_x=0.5,
        xaxis_title="False Positive Rate (1 – Specificity)",
        yaxis_title="True Positive Rate (Recall)",
        xaxis=dict(range=[0,1], showgrid=True),
        yaxis=dict(range=[0,1], showgrid=True),
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=700, height=600
    )

    return fig


In [ ]:
plot_roc_curve_area(train_eval_df, test_eval_df)

## Plot 5

In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_reference_pr_curves():
    p = np.linspace(0, 1, 200)
    perfect_r = np.where(p < 1, 1.0, 0.0)
    L = 0.5
    good_r = 1 - (1 - L)*p**4     # = 1 - 0.75*p**2 → at p=0 → 1, at p=1 → 0.25
    auc_good  = np.trapz(good_r, p)

    fig = go.Figure()

    # ---- Three colored zones (layer='below') ----
    # 1) Above 0.5 → light green
    fig.add_shape(dict(
        type="rect",
        xref="paper", x0=0, x1=1,
        yref="y",     y0=0.5, y1=1,
        fillcolor="rgba(0,255,0,0.2)",
        line=dict(width=0),
        layer="below"
    ))
    # 2) Middle band 0.25–0.5 → light red
    fig.add_shape(dict(
        type="rect",
        xref="paper", x0=0, x1=1,
        yref="y",     y0=0.25, y1=0.5,
        fillcolor="rgba(255,0,0,0.1)",
        line=dict(width=0),
        layer="below"
    ))
    # 3) Below 0.25 → darker red
    fig.add_shape(dict(
        type="rect",
        xref="paper", x0=0, x1=1,
        yref="y",     y0=0, y1=0.25,
        fillcolor="rgba(255,0,0,0.3)",
        line=dict(width=0),
        layer="below"
    ))

    # ---- Dashed baseline lines (layer='above') ----
    fig.add_shape(type="line",
        x0=0, y0=0.5, x1=1, y1=0.5,
        line=dict(dash="dash", color="black"),
        layer="above"
    )
    fig.add_shape(type="line",
        x0=0, y0=0.25, x1=1, y1=0.25,
        line=dict(dash="dash", color="black"),
        layer="above"
    )

    # ---- PR curves ----
    fig.add_trace(go.Scatter(
        x=p, y=perfect_r, mode="lines",
        line=dict(color="Green", width=3),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=p, y=good_r, mode="lines",
        line=dict(color="blue", width=3,
                  shape="spline", smoothing=1.2),
        showlegend=False
    ))

    # ---- On‐curve labels ----
    fig.add_annotation(
        x=0.85, y=0.95, text="AP = 1.00 (Perfect)",
        font=dict(color="Green", size=14),
        showarrow=False,
    )
    fig.add_annotation(
        x=0.75, y=0.7, 
        text=f"AP ≈ {auc_good:.2f} (Good)",
        font=dict(color="blue", size=14),
        showarrow=False,
    )
    fig.add_annotation(
        x=0.15, y=0.5, text="Baseline (P:N=1:1)",
        font=dict(color="black", size=14),
        showarrow=False, yshift=10
    )
    fig.add_annotation(
        x=0.15, y=0.25, text="Baseline (P:N=1:3)",
        font=dict(color="black", size=14),
        showarrow=False, yshift=10
    )
    fig.add_annotation(
        x=0.75, y=0.35, text="Poor (P:N=1:1)",
        font=dict(color="rgba(255,0,0,0.5)", size=14),
        showarrow=False, yshift=10
    )
    fig.add_annotation(
        x=0.75, y=0.15, text="Poor (P:N=1:3)",
        font=dict(color="rgba(255,0,0,0.7)", size=14),
        showarrow=False, yshift=10
    )
    # ---- Axis styling (no grid lines) ----
    fig.update_xaxes(
        title="Recall",
        range=[0,1],
        showgrid=False,
        zeroline=False,
        showline=True,
        linecolor="black",
        linewidth=2
    )
    fig.update_yaxes(
        title="Precision",
        range=[0,1],
        showgrid=False,
        zeroline=False,
        showline=True,
        linecolor="black",
        linewidth=2
    )

    # ---- Final layout ----
    fig.update_layout(
        title={
            "text": "Reference PR Curve",
            "x": 0.5, "xanchor": "center"
        },
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=700, height=600
    )

    return fig

# Usage:
# fig = plot_reference_pr_curves()
# fig.show()


In [ ]:
plot_reference_pr_curves()

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd

def plot_pr_curve_area(train_eval_df: pd.DataFrame,
                       test_eval_df: pd.DataFrame) -> go.Figure:
    """
    Overlay smooth Precision–Recall areas for train and test,
    hide their own legend entries, place the Average Precision (AP)
    and also draw horizontal baseline lines at P/(P+N) for each set,
    labeling them with the P:N ratio.
    """
    def prepare(dfm: pd.DataFrame):
        """
        Expects dfm to have columns: 
        ['threshold','TN','FP','FN','TP','precision','recall']
        Returns:
        dfc, ap_val, baseline, pn_ratio
        """
        # 1) Compute P and N once (they're constant over thresholds)
        #    We can take the first row, or filter threshold==0 if you prefer.
        first = dfm.iloc[0]
        P = first.TP + first.FN
        N = first.TN + first.FP

        # 2) Baseline precision = P / (P + N)
        baseline   = P / (P + N)

        # 3) P:N ratio
        pn_ratio   = P / N

        # 4) Build a PR table
        dfc = dfm[["threshold", "recall", "precision"]].copy()

        # 5) Prepend the (0,1) endpoint if you like
        endpoint = pd.DataFrame({
            "threshold": [1.0],
            "recall":    [0.0],
            "precision": [1.0]
        })
        dfc = pd.concat([dfc, endpoint], ignore_index=True) \
            .drop_duplicates("threshold", keep="last") \
            .sort_values("recall") \
            .reset_index(drop=True)

        # 6) Compute Average Precision via trapezoid
        ap_val = np.trapz(dfc["precision"], dfc["recall"])

        return dfc, ap_val, baseline, pn_ratio

    def corner_for_ap(ap: float):
        # Choose annotation corner
        if ap > 0.5:
            return dict(x=0.4, y=0.05, xanchor="left",  yanchor="bottom")
        else:
            return dict(x=0.75, y=0.95, xanchor="right",   yanchor="top")

    # Prepare train and test
    df_train, ap_train, base_train, pn_train = prepare(train_eval_df)
    df_test,  ap_test,  base_test,  pn_test  = prepare(test_eval_df)

    fig = go.Figure()

    # --- Train PR curve (no legend) ---
    fig.add_trace(go.Scatter(
        x=df_train["recall"], y=df_train["precision"],
        mode="lines",
        line=dict(color="rgb(31,119,180)", shape="spline", smoothing=1.3),
        fill="tozeroy", fillcolor="rgba(31,119,180,0.2)",
        marker=dict(size=6),
        showlegend=False,
        hovertemplate=(
            "Train PR<br>"
            "Threshold %{text}<br>"
            "Recall = %{x:.2f}<br>"
            "Precision = %{y:.2f}<extra></extra>"
        ),
        text=[f"{t:.2f}" for t in df_train["threshold"]]
    ))

    # --- Test PR curve (no legend) ---
    fig.add_trace(go.Scatter(
        x=df_test["recall"], y=df_test["precision"],
        mode="lines",
        line=dict(color="rgb(255,127,14)", shape="spline", smoothing=1.3),
        fill="tozeroy", fillcolor="rgba(255,127,14,0.2)",
        marker=dict(size=6),
        showlegend=False,
        hovertemplate=(
            "Test PR<br>"
            "Threshold %{text}<br>"
            "Recall = %{x:.2f}<br>"
            "Precision = %{y:.2f}<extra></extra>"
        ),
        text=[f"{t:.2f}" for t in df_test["threshold"]]
    ))

    # --- Baseline lines (with legend entries) ---
    # Train baseline
    fig.add_trace(go.Scatter(
        x=[0, 1], y=[base_train, base_train],
        mode="lines",
        line=dict(dash="dot", color="rgb(31,119,180)", width=2),
        name=f"Train baseline (P:N≈{pn_train:.2f}:1)",
        showlegend=False,
    ))
    # Test baseline
    fig.add_trace(go.Scatter(
        x=[0, 1], y=[base_test,  base_test],
        mode="lines",
        line=dict(dash="dash", color="rgb(255,127,14)", width=2),
        name=f"Test baseline (P:N≈{pn_test:.2f}:1)",
        showlegend=False,
    ))

    # ---- AUC annotations (corner) ----
    p_train = corner_for_ap(ap_train)
    fig.add_annotation(
        text=f"<b>Train AP:</b> {ap_train:.2f}",
        showarrow=False,
        font=dict(color="rgb(31,119,180)", size=14),
        bgcolor="rgba(255,255,255,0.7)",
        xref="paper", yref="paper",
        **p_train
    )

    p_test = corner_for_ap(ap_test)
    # if they’d collide, shift only the TEST one vertically:
    if p_test == p_train:
        p_test["y"] += 0.07 if p_test["yanchor"]=="bottom" else -0.07

    fig.add_annotation(
        text=f"<b>Test AP:</b> {ap_test:.2f}",
        showarrow=False,
        font=dict(color="rgb(255,127,14)", size=14),
        bgcolor="rgba(255,255,255,0.7)",
        xref="paper", yref="paper",
        **p_test
    )

    # ---- Baseline annotations (at end of line) ----
    fig.add_annotation(
        x=0.3, y=base_train - 0.05,
        xref="x", yref="y",
        text=f"Train baseline (P:N≈{pn_train:.1f}:1.0)",
        showarrow=False,
        font=dict(color="rgb(31,119,180)", size=12),
        xanchor="right", yanchor="middle",
        # bgcolor="rgba(255,255,255,0.7)"
    )
    fig.add_annotation(
        x=1, y=base_test+0.05,
        xref="x", yref="y",
        text=f"Test baseline (P:N≈{pn_test:.1f}:1.0)",
        showarrow=False,
        font=dict(color="rgb(255,127,14)", size=12),
        xanchor="right", yanchor="middle",
        # bgcolor="rgba(255,255,255,0.7)"
    )

    # --- Layout ---
    fig.update_layout(
        title={
            "text": "Train vs. Test Precision–Recall Curves",
            "x": 0.5, "xanchor": "center"
        },
        title_x=0.5,
        xaxis_title="Recall",
        yaxis_title="Precision",
        xaxis=dict(range=[0,1], showgrid=True),
        yaxis=dict(range=[0,1], showgrid=True),
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=750, height=600
    )

    return fig


In [ ]:
plot_pr_curve_area(train_eval_df, test_eval_df)

## Plot 6

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def plot_decision_threshold_slider(
    dfm: pd.DataFrame,
    metrics: list = ["precision", "recall", "f1", "accuracy", "specificity"],
    default_threshold: float = 0.5
) -> go.Figure:
    """
    Plot several metrics vs. threshold with a slider.
    Only precision & recall are shown initially; the rest start as 'legendonly'.
    Legend‐click will toggle any curve on/off.
    """
    # 1) Sort and find default index
    df = dfm.sort_values("threshold").reset_index(drop=True)
    if default_threshold in df["threshold"].values:
        default_idx = int(df.index[df["threshold"] == default_threshold][0])
    else:
        default_idx = 0

    # 2) Create the figure and metric traces
    colors = {
        "precision":   "blue",
        "recall":      "red",
        "f1":          "green",
        "accuracy":    "purple",
        "specificity": "orange"
    }
    # Only these two should be visible at start:
    default_on = {"precision", "recall"}

    fig = go.Figure()
    for metric in metrics:
        fig.add_trace(go.Scatter(
            x=df["threshold"],
            y=df[metric],
            mode="lines",
            name=metric.capitalize(),
            line=dict(color=colors[metric], shape="spline", smoothing=1.3),
            marker=dict(size=6),
            visible=True if metric in default_on else "legendonly",
            legendgroup=metric.capitalize()
        ))

    # 3) Vertical threshold line
    t0 = df.loc[default_idx, "threshold"]
    fig.add_shape(dict(
        type="line", x0=t0, x1=t0, y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(dash="dot", color="black", width=2)
    ))

    # 4) Single annotation outside the plot
    def make_text(idx):
        lines = []
        for m in metrics:
            val = df.loc[idx, m]
            lines.append(f"<b>{m.capitalize()}:</b> {val:.2f}")
        return "<br>".join(lines)

    fig.update_layout(
        annotations=[dict(
            x=1.02, y=0.02, xref="paper", yref="paper",
            text=make_text(default_idx),
            showarrow=False, align="left",
            font=dict(size=14),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1,
            opacity=0.8
        )],
        margin=dict(l=60, r=160, t=80, b=80)
    )

    # 5) Slider steps: update line + annotation
    steps = []
    for i, row in df.iterrows():
        t = row["threshold"]
        args = {
            "shapes[0].x0": t,
            "shapes[0].x1": t,
            "annotations[0].text": make_text(i)
        }
        steps.append(dict(
            method="relayout",
            label=f"{t:.2f}",
            args=[args]
        ))

    slider = {
        "active": default_idx,
        "pad": {"t": 50},
        "len": 0.8,
        "x": 0.1,
        "steps": steps,
        "currentvalue": {
            "visible": True,
            "prefix": "Threshold: ",
            "xanchor": "center",
            "font": {"size": 14, "color": "black"},
        },
        # hide every tick-mark
        "ticklen":   0,
        "tickwidth": 0,
        "tickcolor": "rgba(0,0,0,0)",
        "font": {"color": "rgba(0,0,0,0)"},  # still hide the labels
        "bgcolor": "white",
        "bordercolor": "lightgray",
    }

    # 6) Final layout
    fig.update_layout(
        sliders=[slider],
        title="Decision Threshold Metrics\n(Precision & Recall shown by default)",
        xaxis=dict(
            title="Threshold", range=[0,1], dtick=0.1,
            showgrid=True, gridcolor="lightgray"
        ),
        yaxis=dict(
            title="Metric Value", range=[0,1], dtick=0.1,
            showgrid=True, gridcolor="lightgray"
        ),
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=900, height=600,
        legend=dict(
            itemclick="toggle",           # click to show/hide each trace
            itemdoubleclick="toggleothers"
        )
    )

    return fig


In [ ]:
plot_decision_threshold_slider(train_eval_df)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_gain_lift_plotly(
    data: pd.DataFrame,
    actual: str = "actual",
    probability: str = "probability",
    bins: int = 10
) -> go.Figure:
    """
    Plot interactive Gain and Lift charts with Plotly, including baseline annotations.
    """
    # Prepare and bucket the data
    df = data[[actual, probability]].copy()
    df = df.sort_values(by=probability, ascending=False).reset_index(drop=True)
    df["bucket"] = pd.qcut(df.index, q=bins, labels=False)

    # Compute bucket stats
    total_pos = df[actual].sum()
    stats = (
        df.groupby("bucket")[actual]
          .agg(bucket_positives="sum", bucket_size="count")
          .sort_index()
    )
    stats["cum_positives"] = stats["bucket_positives"].cumsum()
    stats["gain"] = stats["cum_positives"] / total_pos
    stats["pop_pct"] = (stats.index + 1) / bins
    stats["lift"] = stats["gain"] / stats["pop_pct"]

    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Gain Chart", "Lift Chart"),
        horizontal_spacing=0.15
    )

    # Gain chart: model vs. random baseline
    fig.add_trace(
        go.Scatter(
            x=stats["pop_pct"],
            y=stats["gain"],
            mode="lines+markers",
            name="Model Gain",
            hovertemplate="Population: %{x:.0%}<br>Gain: %{y:.0%}<extra></extra>"
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode="lines",
            name="Random Baseline",
            line=dict(dash="dash", color="gray"),
            hoverinfo="skip",
            showlegend=False
        ),
        row=1, col=1
    )
    # Annotation for the random baseline
    fig.add_annotation(
        x=0.55, y=0.5,
        xref="x1", yref="y1",
        text="Baseline(Random model)",
        showarrow=False,
        font=dict(color="gray", size=12),
        textangle=-43
    )

    # Lift chart: model vs. lift=1 baseline
    fig.add_trace(
        go.Scatter(
            x=stats["pop_pct"],
            y=stats["lift"],
            mode="lines+markers",
            name="Model Lift",
            hovertemplate="Population: %{x:.0%}<br>Lift: %{y:.2f}<extra></extra>"
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=[0, 1],
            y=[1, 1],
            mode="lines",
            name="Lift = 1",
            line=dict(dash="dash", color="gray"),
            hoverinfo="skip",
            showlegend=False
        ),
        row=1, col=2
    )
    # Annotation for the lift=1 baseline
    fig.add_annotation(
        x=0.5, y=1.05,
        xref="x2", yref="y2",
        text="Baseline (Lift=1)",
        showarrow=False,
        font=dict(color="gray", size=12)
    )

    # Axes & layout
    fig.update_xaxes(title="Cumulative Population %", tickformat=".0%", row=1, col=1)
    fig.update_yaxes(title="Cumulative Positives %", tickformat=".0%", row=1, col=1)
    fig.update_xaxes(title="Cumulative Population %", tickformat=".0%", row=1, col=2)
    fig.update_yaxes(title="Lift (Gain / Cum. Pop. %)", row=1, col=2)

    fig.update_layout(
        legend=dict(x=0.35, y=1.15, orientation="h"),
        width=900, height=500,
        template="plotly_white"
    )

    return fig


In [ ]:
plot_gain_lift_plotly(train_df, actual="actual", probability="probability", bins=10)